# Airport Data Aggregator

### This Python script aggregates data from three different sources - departure information, arrival information, and historical weather data - for the airports MCO (Orlando International Airport), ORD (Chicago O'Hare International Airport), and JFK (John F. Kennedy International Airport). It filters and merges the data, ensuring accuracy and completeness, and saves the resulting merged DataFrame to a CSV file named 'arrival_depature_weather_merged_df.csv'.


In [27]:
import pandas as pd

# Load CSV files
departure_df = pd.read_csv('departures.csv')
arrival_df = pd.read_csv('arrivals_syr.csv')
weather_df = pd.read_csv('weather_mco_ord_jfk_syr.csv')

departure_df = departure_df[departure_df['Origin Airport'].isin(['MCO', 'ORD', 'JFK'])]

arrival_df = arrival_df[arrival_df['Origin Airport'].isin(['MCO', 'ORD', 'JFK'])]

print(len(arrival_df),len(departure_df))

23634 17463


In [28]:
# Convert date and time to datetime for accurate merging
departure_df['DateTime'] = pd.to_datetime(departure_df['Date (MM/DD/YYYY)'] + ' ' + departure_df['Scheduled departure time'])

# Function to adjust date and time
def adjust_datetime(date_str, time_str):
    if time_str == '24:00':
        time_str = '00:00'
        date_obj = pd.to_datetime(date_str) + pd.Timedelta(days=1)
        date_str = date_obj.strftime('%m/%d/%Y')
    return date_str, time_str

# Adjust '24:00:00' times in the arrival dataframe
arrival_df[['Date (MM/DD/YYYY)', 'Scheduled Arrival Time']] = arrival_df.apply(
    lambda row: adjust_datetime(row['Date (MM/DD/YYYY)'], row['Scheduled Arrival Time']), axis=1, result_type='expand'
)

In [29]:
# arrival_df['Scheduled Arrival Time'] = arrival_df['Scheduled Arrival Time'].str.replace('24:00:00', '24:00')
arrival_df['DateTime'] = pd.to_datetime(arrival_df['Date (MM/DD/YYYY)'] + ' ' + arrival_df['Scheduled Arrival Time'])
arrival_df = arrival_df.dropna(subset=['DateTime'])
print(len(arrival_df),len(departure_df))

23634 17463


In [30]:
# Function to convert time based on station
def convert_time(row):
    time = pd.to_datetime(row['valid_time_gmt'], unit='s').tz_localize('UTC')
    if row['obs_id'] == 'KSYR' or row['obs_id'] == 'KJFK' or row['obs_id'] == 'KMCO':
        return time.tz_convert('America/New_York').tz_localize(None)
    elif row['obs_id'] == 'KORD':
        return time.tz_convert('America/Chicago').tz_localize(None)

weather_df['valid_time_local'] = weather_df.apply(convert_time, axis=1)


# Columns to be dropped
columns_to_drop = [
    'primary_wave_period', 'primary_wave_height', 'primary_swell_period',
    'primary_swell_height', 'primary_swell_direction', 'secondary_swell_period',
    'secondary_swell_height', 'secondary_swell_direction', 'qualifier',
    'qualifier_svrty', 'blunt_phrase', 'terse_phrase', 'max_temp', 'min_temp',
   'pressure_tend', 'pressure_desc','water_temp','wdir','key','class','obs_name','gust'
]

# Drop the specified columns, assuming they exist in the sample data
# For demonstration, we'll check if these columns exist in weather_df and ignore if they don't
columns_to_drop = [col for col in columns_to_drop if col in weather_df.columns]

# Dropping the columns from weather_df
weather_df = weather_df.drop(columns=columns_to_drop)

# Set NaN values to 0 for specific columns
columns_to_fill_zero = ['snow_hrly', 'precip_total', 'gust']

for col in columns_to_fill_zero:
    if col in weather_df.columns:
        weather_df[col] = weather_df[col].fillna(0)

# Assuming the provided sample row represents the typical structure of the weather_df DataFrame
# and using it to demonstrate how to fill NaN values

# For numerical columns, we'll use the median to fill NaN values
numerical_columns = ['temp', 'wx_icon', 'icon_extd', 'dewPt', 'heat_index', 'rh', 
                     'pressure', 'vis', 'wc', 'wspd', 'feels_like', 'uv_index']

for col in numerical_columns:
    if col in weather_df.columns:  # Check if the column exists in the DataFrame
        weather_df[col] = weather_df[col].fillna(weather_df[col].median())

# For categorical columns, we'll use the mode to fill NaN values
categorical_columns = ['wx_phrase', 'wdir_cardinal', 'uv_desc', 'clds']

for col in categorical_columns:
    if col in weather_df.columns:  # Check if the column exists in the DataFrame
        weather_df[col] = weather_df[col].fillna(weather_df[col].mode().iloc[0])


syr_df = weather_df[weather_df['obs_id']=='KSYR']
mco_df = weather_df[weather_df['obs_id']=='KMCO']
ord_df = weather_df[weather_df['obs_id']=='KORD']
jfk_df = weather_df[weather_df['obs_id']=='KJFK']

In [31]:
# Sort the dataframes by time
arrival_df = arrival_df.sort_values(by='DateTime')
syr_df = syr_df.sort_values(by='valid_time_local')

departure_df = departure_df.sort_values(by='DateTime')
mco_df = mco_df.sort_values(by='valid_time_local')
jfk_df = jfk_df.sort_values(by='valid_time_local')
ord_df = ord_df.sort_values(by='valid_time_local')

# Merge for arrivals with suffixes
matched_arrivals = pd.merge_asof(
    arrival_df, syr_df, left_on='DateTime', right_on='valid_time_local', direction='backward', suffixes=('_a', '_w'))

# Merge for departures with suffixes
matched_departures_mco = pd.merge_asof(
    departure_df[departure_df['Origin Airport'] == 'MCO'], mco_df, left_on='DateTime', right_on='valid_time_local', direction='backward', suffixes=('_d', '_w'))
matched_departures_jfk = pd.merge_asof(
    departure_df[departure_df['Origin Airport'] == 'JFK'], jfk_df, left_on='DateTime', right_on='valid_time_local', direction='backward', suffixes=('_d', '_w'))
matched_departures_ord = pd.merge_asof(
    departure_df[departure_df['Origin Airport'] == 'ORD'], ord_df, left_on='DateTime', right_on='valid_time_local', direction='backward', suffixes=('_d', '_w'))

matched_departures = pd.concat([matched_departures_mco, matched_departures_jfk, matched_departures_ord])

print(len(arrival_df)== len(matched_arrivals))
print(len(departure_df) == len(matched_departures))
print(len(matched_arrivals))
print(len(matched_departures))

True
True
23634
17463


In [39]:
join_columns = ['Carrier Code', 'Origin Airport', 'Flight Number', 'Tail Number', 'Date (MM/DD/YYYY)']

# Remove duplicates from each dataframe based on the join columns
matched_arrivals_unique = matched_arrivals.drop_duplicates(subset=join_columns, keep='first')
matched_departures_unique = matched_departures.drop_duplicates(subset=join_columns, keep='first')

# Append suffixes to all columns except the join columns in both dataframes
matched_arrivals_unique = matched_arrivals_unique.rename(columns={col: f"{col}_a" for col in matched_arrivals_unique.columns if col not in join_columns})
matched_departures_unique = matched_departures_unique.rename(columns={col: f"{col}_d" for col in matched_departures_unique.columns if col not in join_columns})

# Now, rename the join columns to include the suffixes for the merge
for col in join_columns:
    matched_arrivals_unique.rename(columns={col: f"{col}_a"}, inplace=True)
    matched_departures_unique.rename(columns={col: f"{col}_d"}, inplace=True)

# Merge on the renamed join columns
merged_df = pd.merge(
    matched_arrivals_unique, 
    matched_departures_unique, 
    left_on=[f"{col}_a" for col in join_columns],
    right_on=[f"{col}_d" for col in join_columns],
    how='inner'
)

print(len(merged_df))

17463


In [40]:
print(merged_df.columns)

Index(['Carrier Code_a', 'Date (MM/DD/YYYY)_a', 'Flight Number_a',
       'Tail Number_a', 'Origin Airport_a', 'Scheduled Arrival Time_a',
       'Actual Arrival Time_a', 'Scheduled Elapsed Time (Minutes)_a',
       'Actual Elapsed Time (Minutes)_a', 'Arrival Delay (Minutes)_a',
       'Wheels-on Time_a', 'Taxi-In time (Minutes)_a',
       'Delay Carrier (Minutes)_a', 'Delay Weather (Minutes)_a',
       'Delay National Aviation System (Minutes)_a',
       'Delay Security (Minutes)_a', 'Delay Late Aircraft Arrival (Minutes)_a',
       'DateTime_a', 'Unnamed: 0_a', 'expire_time_gmt_a', 'obs_id_a',
       'valid_time_gmt_a', 'day_ind_a', 'temp_a', 'wx_icon_a', 'icon_extd_a',
       'wx_phrase_a', 'dewPt_a', 'heat_index_a', 'rh_a', 'pressure_a', 'vis_a',
       'wc_a', 'wdir_cardinal_a', 'wspd_a', 'precip_total_a', 'precip_hrly_a',
       'snow_hrly_a', 'uv_desc_a', 'feels_like_a', 'uv_index_a', 'clds_a',
       'est_time_a', 'est_date_a', 'valid_time_local_a', 'Unnamed: 0_d_d',
       'in

In [41]:
merged_df.to_csv('arrival_depature_weather_merged_df.csv', index=False)